In [ ]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.2'
spark_version = 'spark-3.2.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [947 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelea

In [ ]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-10-05 00:26:59--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  6.01MB/s    in 0.2s    

2022-10-05 00:27:00 (6.01 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge-pet-product-review").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
url = "https://leonardo-bucket.s3.amazonaws.com/vine_table.csv"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("vine_table.csv"), sep=",", header=True, inferSchema=True)
df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| REAKC26P07MDN|          5|            0|          0|   N|                Y|
|R3NU7OMZ4HQIEG|          2|            0|          1|   N|                Y|
|R14QJW3XF8QO1P|          5|            0|          0|   N|                Y|
|R2HB7AX0394ZGY|          5|            0|          0|   N|                Y|
| RGKMPDQGSAHR3|          5|            0|          0|   N|                Y|
|R1DJCVPQGCV66E|          5|            0|          0|   N|                Y|
|R3V52EAWLPBFQG|          3|            0|          0|   N|                Y|
|R3DKO8J1J28QBI|          2|            0|          0|   N|                Y|
| R764DBXGRNECG|          5|            1|          1|   N|                N|
| RW1853GAT0Z9F|          5|            0|          0|   N|     

In [ ]:
df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)



## step 1

Filter the data and create a new DataFrame or table to retrieve all the rows where the total_votes count is equal to or greater than 20 to pick reviews that are more likely to be helpful and to avoid having division by zero errors later on.

In [ ]:
## select all the rows (exept review_id)
df_reviews = df.select(["star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])

#df_reviews.show()

## filter df_reviews for all rows where total_votes are >=20
df_reviews_20 = df_reviews.filter(df_reviews["total_votes"] >=20)

df_reviews_20.show(10)

+-----------+-------------+-----------+----+-----------------+
|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-----------+-------------+-----------+----+-----------------+
|          1|           27|         31|   N|                Y|
|          5|           25|         25|   N|                Y|
|          3|           29|         31|   N|                Y|
|          2|           35|         42|   N|                Y|
|          4|           27|         28|   N|                Y|
|          5|           62|         64|   N|                N|
|          2|           36|         43|   N|                Y|
|          5|           20|         20|   N|                Y|
|          1|           20|         23|   N|                Y|
|          5|           35|         36|   N|                Y|
+-----------+-------------+-----------+----+-----------------+
only showing top 10 rows



## Step 2

Filter the new DataFrame or table created in Step 1 and create a new DataFrame or table to retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%.

In [ ]:
## filter df_reviews_20 for all rows where helpful_votes / total_votes >= 0.5
df_reviews_50 = df_reviews_20.filter(df_reviews_20["helpful_votes"]/df_reviews_20["total_votes"] >= 0.50)

df_reviews_50.show(5)

+-----------+-------------+-----------+----+-----------------+
|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-----------+-------------+-----------+----+-----------------+
|          1|           27|         31|   N|                Y|
|          5|           25|         25|   N|                Y|
|          3|           29|         31|   N|                Y|
|          2|           35|         42|   N|                Y|
|          4|           27|         28|   N|                Y|
+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



## Step 3

Filter the DataFrame or table created in Step 2, and create a new DataFrame or table that retrieves all the rows where a review was written as part of the Vine program (paid), vine == 'Y'.

In [ ]:
# filter df_reviews_50 for all rows where vine = "Y"
paid_reviews_df = df_reviews_50.filter(df_reviews_50["vine"] == "Y")

paid_reviews_df.show(5)

+-----------+-------------+-----------+----+-----------------+
|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-----------+-------------+-----------+----+-----------------+
|          2|           27|         30|   Y|                N|
|          5|           72|         72|   Y|                N|
|          5|           39|         42|   Y|                N|
|          5|           29|         30|   Y|                N|
|          2|           20|         20|   Y|                N|
+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [ ]:
paid_reviews_df.describe().show(5)

+-------+------------------+------------------+------------------+----+-----------------+
|summary|       star_rating|     helpful_votes|       total_votes|vine|verified_purchase|
+-------+------------------+------------------+------------------+----+-----------------+
|  count|               170|               170|               170| 170|              170|
|   mean| 3.929411764705882|63.641176470588235| 68.03529411764706|null|             null|
| stddev|1.1124708391270801|107.53304464635706|110.81862649267784|null|             null|
|    min|                 1|                13|                20|   Y|                N|
|    max|                 5|              1239|              1277|   Y|                Y|
+-------+------------------+------------------+------------------+----+-----------------+



# Step 4

Repeat Step 3, but this time retrieve all the rows where the review was not part of the Vine program (unpaid), vine == 'N'.

In [ ]:
# filter df_reviews_50 for all rows where vine = "Y"
unpaid_reviews_df = df_reviews_50.filter(df_reviews_50["vine"] == "N")

unpaid_reviews_df.show(5)


+-----------+-------------+-----------+----+-----------------+
|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-----------+-------------+-----------+----+-----------------+
|          1|           27|         31|   N|                Y|
|          5|           25|         25|   N|                Y|
|          3|           29|         31|   N|                Y|
|          2|           35|         42|   N|                Y|
|          4|           27|         28|   N|                Y|
+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [ ]:
unpaid_reviews_df.describe().show(5)

+-------+------------------+------------------+------------------+-----+-----------------+
|summary|       star_rating|     helpful_votes|       total_votes| vine|verified_purchase|
+-------+------------------+------------------+------------------+-----+-----------------+
|  count|             37840|             37840|             37840|37840|            37840|
|   mean|3.7647463002114163| 57.50052854122622|61.863583509513745| null|             null|
| stddev|1.6055561346315663|107.60541722886983|111.92734632290345| null|             null|
|    min|                 1|                10|                20|    N|                N|
|    max|                 5|              5735|              5811|    N|                Y|
+-------+------------------+------------------+------------------+-----+-----------------+



# Step 5

Determine the total number of reviews, the number of 5-star reviews, and the percentage of 5-star reviews for the two types of review (paid vs unpaid).

## For Paid (Vine) Reviews

In [ ]:
# determine count of paid_5star_reviews
paid_5start_reviews = paid_reviews_df[paid_reviews_df["star_rating"] == 5].count()
paid_5start_reviews

65

In [ ]:
# determine count of paid reviews
paid_reviews = paid_reviews_df.count()
paid_reviews

170

In [ ]:
# determine percentage of paid_5star_reviews
percentage_paid_5star_reviews = paid_5start_reviews / paid_reviews *100
percentage_paid_5star_reviews

38.23529411764706

## For Unpaid (Non-Vine) Reviews

In [ ]:
# determine count of paid_5star_reviews
unpaid_5start_reviews = unpaid_reviews_df[unpaid_reviews_df["star_rating"] == 5].count()
unpaid_5start_reviews

20612

In [ ]:
# determine count of paid reviews
unpaid_reviews = unpaid_reviews_df.count()
unpaid_reviews

37840

In [ ]:
# determine percentage of paid_5star_reviews
percentage_unpaid_5star_reviews = unpaid_5start_reviews / unpaid_reviews *100
percentage_unpaid_5star_reviews

54.471458773784356

# Analysis:

Ho: 5star_vine = 5star_not_vine (no difference in the percenage(proportion) of 5star ratings paid(vine) vs unpaid(not vine).

Ha: 5star_vine <> 5star_not_vine(there is a difference).

- percentage_5star_vine = **38.24%**
- percentage_5star_not_vine = **54.47%**
- **findings: the percentage of 5star rating is not the same when comparing vine reviewsto not_vine reviews.**
- **There is reason to believe that the percentage of 5star raitings is not directly tied to whether the review is paid or unpaid.**
- note: this hypo thesis testwould need tobe evaluated comparing the proportion of 5star reviews among the two samples.